In [ ]:
!pip install unstructured
!pip install tiktoken
!pip install pypdf

In [ ]:
long_text = """

1. Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

2. Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.

3. Overall, Assistant is a powerful tool that can help with a wide range of tasks and provide valuable insights and information on a wide range of topics. Whether you need help with a specific question or just want to have a conversation about a particular topic, Assistant is here to assist.

4. Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

5. Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.

6. Overall, Assistant is a powerful tool that can help with a wide range of tasks and provide valuable insights and information on a wide range of topics. Whether you need help with a specific question or just want to have a conversation about a particular topic, Assistant is here to assist.

"""

CharacterTextSplitter

In [ ]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(        
    separator = "\n\n",
    chunk_size = 50,
    chunk_overlap  = 10,
    length_function = len,
)

documents = text_splitter.create_documents([long_text])
print(documents[0].page_content)
print(documents[1].page_content)

RecursiveCharacterTextSplitter

TokenTextSplitter

In [ ]:
from langchain.text_splitter import TokenTextSplitter

text_splitter = TokenTextSplitter(chunk_size=50, chunk_overlap=0)
documents = text_splitter.create_documents([long_text])
print(documents[0])

PyPDFLoader: PDF文档加载器

RecursiveCharacterTextSplitter: 文档分割器，以递归方式按照给定的字符切分文档.

load_summarize_chain: 总结给定的文档

In [ ]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader(r"C:\Users\chenyzhang\Downloads\test.pdf")
docs = loader.load()
print(len(pages))

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0) # chunk_overlap文档分割时上下文重叠的字符数
docs = text_splitter.split_documents(docs)
print(len(docs))
print(docs[0].page_content)

In [ ]:
from langchain.llms import OpenAI
from langchain.chains.summarize import load_summarize_chain

llm = OpenAI(temperature=0.5, model_name="text-davinci-003")
chain = load_summarize_chain(llm, chain_type="map_reduce", verbose=False)

print(chain)

In [ ]:
from langchain.callbacks import get_openai_callback

with get_openai_callback() as cb:
    response = chain.run(input_documents=docs[:2])
    print(f"Total Tokens: {cb.total_tokens}")
    print(f"Prompt Tokens: {cb.prompt_tokens}")
    print(f"Completion Tokens: {cb.completion_tokens}")
    print(f"Successful Requests: {cb.successful_requests}")
    print(f"Total Cost (USD): ${cb.total_cost}")
    print(response)

Chromadb: Chroma是一种VectorStores，用来保存和检索向量

Chroma开源、轻量、性能良好，使用起来也很简单，只需使用pip安装chromadb即可。

In [ ]:
!pip install chromadb

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

embeddings = OpenAIEmbeddings()
persist_directory = "chroma_storage"
vectorstore = Chroma.from_documents(docs, embeddings, persist_directory=persist_directory)

In [ ]:
print(docs[1].page_content)
query = '购买新能源汽车的主要原因是什么？'

In [ ]:
similarity_docs = vectorstore.similarity_search(query)
print("similarity_docs number: ", len(similarity_docs))
print("characters number: ", sum(len(item.page_content) for item in similarity_docs))
print(similarity_docs[0].page_content)

In [ ]:
import tiktoken

encoding = tiktoken.encoding_for_model(llm.model_name) # tiktoken.get_encoding("cl100k_base")
print(len(encoding.encode(''.join(item.page_content for item in similarity_docs))))
print(len(encoding.encode(''.join(item.page_content for item in similarity_docs[:1]))))

In [ ]:
vectorstore.persist()
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embeddings)

similarity_docs = vectordb.similarity_search(query) # include_metadata=True
print("similarity_docs number: ", len(similarity_docs))
print("characters number: ", sum(len(item.page_content) for item in similarity_docs))
print(similarity_docs[0].page_content)

In [52]:
from langchain.chains.question_answering import load_qa_chain

chain = load_qa_chain(llm, chain_type="stuff") # chain_type="map_reduce", chain_type="refine"
chain.run(input_documents=similarity_docs[:1], question=query)

' 牌照问题是最初购买新能源汽车的首要考虑因素，三成受访者未来会再购买新能源汽车，经济实用则是未来购买的主要动力。'